In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import six
import collections
import random
import time

In [3]:
#import config
from collections import defaultdict

In [4]:
def user_sequence(fname):
    usernum = 0
    itemnum = 0
    Item = defaultdict(list)
    item_train = {}
    item_valid = {}
    item_test = {}
    
    user_num = []
    item_num = []
    
    # assume user/item index starting from 1
    f = open(fname, 'r')
    for line in f:
        i, u = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        Item[i].append(u)
    
        user_num.append(u)
        item_num.append(i)
    
    user_num = list(dict.fromkeys(user_num))
    item_num = list(dict.fromkeys(item_num))
    
    for item in Item:
        nfeedback = len(Item[item])
        if nfeedback < 3:
            item_train[item] = Item[item]
            item_valid[item] = []
            item_test[item] = []
        else:
            item_train[item] = Item[item][:-2]
            item_valid[item] = []
            item_valid[item].append(Item[item][-2])
            item_test[item] = []
            item_test[item].append(Item[item][-1])
    return [item_train, item_valid, item_test, len(user_num), len(item_num)]

def item_sequence(fname):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train = {}
    user_valid = {}
    user_test = {}
    
    user_num = []
    item_num = []
    
    # assume user/item index starting from 1
    f = open(fname, 'r')
    for line in f:
        u, i = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        User[u].append(i)
    
        user_num.append(u)
        item_num.append(i)
    
    user_num = list(dict.fromkeys(user_num))
    item_num = list(dict.fromkeys(item_num))
    
    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:
            user_train[user] = User[user][:-2]
            user_valid[user] = []
            user_valid[user].append(User[user][-2])
            user_test[user] = []
            user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, len(user_num), len(item_num)]

In [6]:
dataset_user_seq = user_sequence('./HybridBERT4Rec/data/ml-1m-item.txt') 
dataset_item_seq = item_sequence('./HybridBERT4Rec/data/ml-1m-item.txt')

In [8]:
[item_train, item_valid, item_test, usernum, itemnum] = dataset_user_seq

In [9]:
cc = 0.0
max_len = 0
min_len = 100000


for i in item_train:
    cc += len(item_train[i])
    max_len = max(len(item_train[i]), max_len)
    min_len = min(len(item_train[i]), min_len)
    # if(min_len==4):
    #     print(u)
    #     print(user_train[u])
    #     break


print('average sequence length: %.2f' % (cc / len(item_train)))
print('max:{}, min:{}'.format(max_len, min_len))

print('len_train:{}, len_valid:{}, len_test:{}, usernum:{}, itemnum:{}'.
    format(
    len(item_train),
    len(item_valid), len(item_test), usernum, itemnum))

average sequence length: 268.00
max:3426, min:1
len_train:3706, len_valid:3706, len_test:3706, usernum:6040, itemnum:3706


In [ ]:
# put validate into train
for i in item_train:
    if i in item_valid:
        item_train[i].extend(item_valid[i])

In [10]:
from HybridBERT4Rec.vocab import *

In [13]:
vocab = FreqVocab(item_train)  # ใช้ vocab แค่ item_test_data เพราะมันมีข้อมูลของทั้ง train และ test

In [14]:
vocab.get_item_count()

6040

In [17]:
import random

In [26]:

# get the max index of the data
item_train_data = {
    str(k): [str(user) for user in v]
    for k, v in item_train.items() if len(v) > 0
}
item_test_data = {
    str(i):
        [str(user) for user in (item_train[i] + item_test[i])]
    # for i in item_train if len(item_train[i]) > 0 and len(item_test[i]) > 0
    for i in item_train if len(item_train[i]) > 0
}
random_seed = 12345
rng = random.Random(random_seed)

vocab = FreqVocab(item_test_data)  # ใช้ vocab แค่ item_test_data เพราะมันมีข้อมูลของทั้ง train และ test

In [28]:
vocabulory = vocab.get_items()
random.shuffle(vocabulory)

In [29]:
vocabulory_lines = "\n".join(vocabulory)

In [31]:
with open('HybridBERT4Rec/vocab.txt','w') as f:
    f.write(vocabulory_lines)

In [32]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMaskedLM, BertConfig

In [40]:
from transformers import BertConfig

In [34]:
# Step 1: Create and load your custom tokenizer with the 6040 vocabulary
custom_vocab_path = "HybridBERT4Rec/vocab.txt"
tokenizer = BertTokenizer.from_pretrained(custom_vocab_path)

# # Step 2: Load the corresponding pre-trained model for the custom vocabulary
# model = TFBertForMaskedLM.from_pretrained("bert-base-uncased")
# # Now you can use the tokenizer and model as usual for training or fine-tuning

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [53]:
# Step 2: Load the pre-trained model and get its configuration
model_name = "bert-base-uncased"
model = TFBertForMaskedLM.from_pretrained(model_name)
config = model.config

# Step 3: Update the model configuration with the custom vocabulary size
config.vocab_size = len(tokenizer)

# Step 4: Load the model again with the updated configuration
model = TFBertForMaskedLM.from_config(config=config)

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [55]:
all_vals = []
for k, val in item_train_data.items():
    all_vals.append(" ".join(val))

In [58]:
max_length = 200

# Tokenize the dataset
tokenized_dataset = tokenizer(all_vals, padding="max_length", truncation=True, max_length=max_length, return_tensors='tf')

In [68]:
#tokenized_dataset['input_ids'].numpy()

# Create labels by copying input_ids
labels = tokenized_dataset["input_ids"].numpy().copy()


In [148]:
mask_probability = 0.15
tokenized_dataset_input_ids = tokenized_dataset["input_ids"].numpy().copy()
for i, row in enumerate(tokenized_dataset["attention_mask"].numpy()):
    # actual tokens
    actual_tokens = tokenized_dataset_input_ids[i]
    cls_token_pos = np.argwhere((actual_tokens == tokenizer.cls_token_id))
    sep_token_pos = np.argwhere((actual_tokens == tokenizer.sep_token_id))
    
    # FINDING indices with non-padding tokens, CLS token and SEP token
    non_pad_ids = np.argwhere(row == 1).flatten()     
    non_pad_ids_list = non_pad_ids.tolist()
    non_pad_ids_list.remove(cls_token_pos)
    non_pad_ids_list.remove(sep_token_pos)
    non_pad_ids = np.array(non_pad_ids_list)
    random.shuffle(non_pad_ids)
    mask_indices = non_pad_ids[:num_mask_tokens]

    num_mask_tokens = int(len(non_pad_ids) * mask_probability)
    
    # Replace masked tokens with [MASK] token id
    tokenized_dataset_input_ids[i][mask_indices] = tokenizer.mask_token_id

tokenized_dataset_input_ids_tensors = tf.convert_to_tensor(tokenized_dataset_input_ids)
tokenized_dataset['input_ids'] = tokenized_dataset_input_ids_tensors

In [155]:
inputs = (tokenized_dataset['input_ids'], tokenized_dataset["attention_mask"])

In [ ]:

# # Main function to train the model
# def main():
#     # Step 1: Load the tokenizer and model
#     tokenizer, model = load_tokenizer_and_model()

#     # Step 2: Prepare your custom dataset for masked language modeling
#     dataset_path = 'path/to/your/custom_dataset.txt'
#     inputs, labels = prepare_dataset(dataset_path, tokenizer)

#     # Step 3: Define the loss function and compile the model
#     model = compile_model(model)

#     # Step 4: Fine-tune the model on your custom dataset
#     batch_size = 16
#     num_epochs = 3
#     model = train_model(model, inputs, labels, batch_size, num_epochs)

#     # Step 5: Save the trained model and tokenizer
#     save_dir = 'path/to/save/model/'
#     save_model_and_tokenizer(model, tokenizer, save_dir)

# if __name__ == "__main__":
#     main()


In [159]:
model_C = compile_model(model)

In [162]:
tensorboard_callback = tf.keras.callbacks.TensorBoard('logdIR', histogram_freq=1)

In [163]:
history = model.fit(
        inputs,
        labels,
        batch_size=8,
        epochs=10,
    callbacks=[tensorboard_callback]
    )

Epoch 1/10
 70/464 [===>..........................] - ETA: 1:04:35 - loss: 0.9722

KeyboardInterrupt: 

In [164]:
model.summary()

Model: "tf_bert_for_masked_lm_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  90092544  
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  5636252   
                                                                 
Total params: 90690716 (345.96 MB)
Trainable params: 90690716 (345.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [158]:
# Step 3: Define the loss function and compile the model
def compile_model(model):
    loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    model.compile(optimizer=optimizer, loss=loss_function)
    return model

# Step 4: Training the model
def train_model(model, inputs, labels, batch_size=16, num_epochs=3):
    model.fit(
        inputs,
        labels,
        batch_size=batch_size,
        epochs=num_epochs,
    )
    return model

# Step 5: Save the trained model and tokenizer
def save_model_and_tokenizer(model, tokenizer, save_dir):
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
